In [ ]:
# !git clone https://github.com/huggingface/transformers.git

In [ ]:
%%capture
!pip install datasets evaluate transformers[sentencepiece]
!pip install rouge_score
!pip install underthesea
!pip install pyvi

In [ ]:
%%capture
!pip install gdown
import gdown

!gdown 1jWDnP1xF01_pmeJYsRHxSVBkLZ0I9LAt #utils T5 for VQA
!gdown 1jHGEuwjMI_P81Ha-L3o5IK6hAu6YQAjG #easyocr
!gdown 1WKJC33RpxTot6YRryVwBuQCRhkehsM2g #scoring
!unzip -o scoring_program.zip

In [ ]:
import json
f = open('/kaggle/input/vlsp2023-dsvivrc/training-data/training-data/vlsp2023_train_data.json')
train_json = json.load(f)
f = open('/kaggle/input/vlsp2023-dsvivrc/dev-set/dev-set/vlsp2023_dev_data.json')
dev_json = json.load(f)
f = open('/kaggle/working/easyocr.json')
easyocr_json = json.load(f)

In [ ]:
#%%capture
%cd /kaggle/working
import os
import shutil

source_all = ['/kaggle/input/vlsp2023-dsvivrc/training-data/training-data/training-images/training-images',
              '/kaggle/input/vlsp2023-dsvivrc/dev-set/dev-set/dev-images/dev-images']
destination = './images'
os.mkdir(destination)

for source in source_all:
    allfiles = os.listdir(source)
    for f in allfiles:
        src_path = os.path.join(source, f)
        dst_path = os.path.join(destination, f)
        shutil.copy(src_path, dst_path)

In [ ]:
import pandas as pd

train_set = pd.DataFrame(train_json['annotations']).T
dev_set = pd.DataFrame(dev_json['annotations']).T

train_set = train_set[['question','answer','image_id']]
train_set['image_id'] = train_set['image_id'].astype(str)
dev_set = dev_set[['question','answer','image_id']]
dev_set['image_id'] = dev_set['image_id'].astype(str)

train_set['question'] = train_set['question'].str.replace(' ?','') + " "+ train_set['image_id'].apply(lambda x: easyocr_json[x])
dev_set['question'] = dev_set['question'].str.replace(' ?','') + " "+ dev_set['image_id'].apply(lambda x: easyocr_json[x])

img_dict_train = train_json['images']
img_dict_dev = dev_json['images']
img_dict = dict(img_dict_train)
img_dict.update(img_dict_dev)

print(len(img_dict_train))
print(len(img_dict_dev))
print(len(img_dict))

In [ ]:
# from PIL import Image
# import tqdm
# from transformers import (
#     AutoTokenizer, AutoFeatureExtractor,
#     AutoModel,            
#     TrainingArguments,
#     logging
# )
# import torch

# #google/vit-base-patch16-224-in21k
# #google/vit-large-patch32-384
# model_name = 'google/vit-base-patch16-224-in21k'
# #model_name ='microsoft/beit-large-patch16-224'
# image_encoder = AutoModel.from_pretrained(model_name)
# preprocessor = AutoFeatureExtractor.from_pretrained(model_name)

# img_w = {}
# for k,v in tqdm.tqdm_notebook(img_dict.items()):
#     imgl = '/kaggle/working/images/' + v
#     processed_images = preprocessor(images=[Image.open(imgl).convert('RGB')],
#                                     return_tensors="pt",)['pixel_values']
#     with torch.no_grad():
#         encoded_image = image_encoder(pixel_values=processed_images, return_dict=True).last_hidden_state[:,1:,:].squeeze()# we discard the CLS token
#         #encoded_image = encoded_image.last_hidden_state[:,1:,:].squeeze() 
#     img_w[k] = encoded_image#.to('cpu')
#     del encoded_image
#     torch.cuda.empty_cache()
# torch.save(img_w, '/kaggle/working/vit-b.pt') # export for later used


# # imgl = '/kaggle/working/images/' + img_dict[int(train_set.image_id[45])]
# # processed_images = preprocessor(images=[Image.open(imgl).convert('RGB')],
# #                                 return_tensors="pt",)['pixel_values']
# # encoded_image = image_encoder(pixel_values=processed_images,
# #                               return_dict=True)

# # last_hidden_state = encoded_image.last_hidden_state[:,1:,:] # we discard the CLS token
# # last_hidden_state.squeeze().size()

In [ ]:
# from IPython.display import FileLink

# FileLink(r'vit-b.pt')

load image features extracted from ViT model

In [ ]:
import torch

#img_w = torch.load('/kaggle/input/vlsp2023-vivrc-beit/beit.pt') # already-saved features
img_w = torch.load('/kaggle/input/vlsp2023-vivrc-vitb/vit-b.pt') # already-saved features
len(img_w)

In [ ]:
from transformers import AutoTokenizer
from utils import T5ForConditionalGeneration
tokenizer = AutoTokenizer.from_pretrained("VietAI/vit5-base")
model = T5ForConditionalGeneration.from_pretrained("VietAI/vit5-base").to('cuda')
model.add_imgw(img_w)

In [ ]:
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainer, TrainingArguments, Seq2SeqTrainingArguments
from tqdm.notebook import tqdm
from torch.utils.data import DataLoader

def preprocess_function(examples):
    model_inputs = tokenizer(
        examples["inputs"], max_length=128, truncation=True, padding=True
    )
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["labels"], max_length=50, truncation=True, padding=True
        )
    model_inputs['labels'] = labels['input_ids']
    model_inputs['input_ids'] = model_inputs['input_ids']
    model_inputs["image_id"] = examples["image_id"]
    
    return model_inputs

In [ ]:
dict_obj = {}
dict_obj['inputs'] = train_set['question'] #[v['question'].replace(' ?','') for _,v in train_json['annotations'].items()]
dict_obj['labels'] =  train_set['answer'] #[v['answer'] for _,v in train_json['annotations'].items()]
dict_obj['image_id'] = train_set['image_id']
train_dataset = Dataset.from_dict(dict_obj)
tokenized_train_datasets = train_dataset.map(preprocess_function, batched=True, remove_columns=['inputs'], num_proc=8)

dict_obj = {}
dict_obj['inputs'] = dev_set['question']#[v['question'].replace(' ?','') for _,v in dev_json['annotations'].items()]
dict_obj['labels'] = dev_set['answer']#[v['answer'] for _,v in dev_json['annotations'].items()]
dict_obj['image_id'] = dev_set['image_id']
dev_dataset = Dataset.from_dict(dict_obj)
tokenized_dev_datasets = dev_dataset.map(preprocess_function, batched=True, remove_columns=['inputs'], num_proc=8)

In [ ]:
import os
from transformers.optimization import Adafactor, AdafactorSchedule
from utils import DataCollatorForSeq2Seq

os.environ["WANDB_DISABLED"] = "True"

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="pt")

#Adam
training_args = Seq2SeqTrainingArguments(output_dir="./checkpoint",
                                      do_train=True,
                                      do_eval=True,
                                      num_train_epochs=30,
                                      learning_rate=2.5e-5,
                                      warmup_ratio=0.05,
                                      weight_decay=0.01,
                                      per_device_train_batch_size=32,
                                      per_device_eval_batch_size=32,
                                      logging_dir='./log',
                                      group_by_length=True,
                                      save_strategy="steps",
                                      save_total_limit=5,
                                      eval_steps=100,
                                      logging_steps = 100,
                                      evaluation_strategy="steps",
                                      save_steps=100,
                                      load_best_model_at_end= True,
                                      fp16=True,
                                      seed=42,
                                      )

# AdaFactor for ViT5-large models as it based on T5v1.1.
# See https://medium.com/the-artificial-impostor/paper-adafactor-adaptive-learning-rates-with-sublinear-memory-cost-a543abffa37
# 
# from transformers.optimization import Adafactor, AdafactorSchedule
# optimizer = Adafactor(
#     model.parameters(),
#     lr=1e-3,
#     eps=(1e-30, 1e-3),
#     clip_threshold=1.0,
#     decay_rate=-0.8,
#     beta1=None,
#     weight_decay=0.0,
#     relative_step=False,
#     scale_parameter=False,
#     warmup_init=False
# )
# lr_scheduler = AdafactorSchedule(optimizer)

In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_datasets,
    eval_dataset=tokenized_dev_datasets,
    data_collator=data_collator,
    #optimizers=(optimizer, lr_scheduler),
)

trainer.train()

In [ ]:
os.listdir('/kaggle/working/checkpoint')

In [ ]:
torch.cuda.empty_cache()
model = T5ForConditionalGeneration.from_pretrained("/kaggle/working/checkpoint/checkpoint-5300")#5300:2.9312
model.to('cuda')
model.add_imgw(img_w)

In [ ]:
from datasets import load_metric
metric = load_metric("rouge")

In [ ]:
from underthesea import word_tokenize
from underthesea import text_normalize
from pyvi import ViUtils

def postprocessing(t):
    words = t.split()
    words_not = words[:int(len(words)/3)]
    words_dup = words[int(len(words)/3):]
    words_dup = sorted(set(words_dup), key=words_dup.index)
    
    words = " ".join(words_not+words_dup)
    words = text_normalize(words)
    #words = ViUtils.add_accents(words)
    return words.lower()

In [ ]:
import torch 
import numpy as np
metrics = load_metric('rouge')

max_target_length = 30
dataloader = torch.utils.data.DataLoader(tokenized_dev_datasets, collate_fn=data_collator, batch_size=32)

predictions = []
references = []

for i, batch in enumerate(tqdm(dataloader)):
    outputs = model.generate(image_id = batch['image_id'],
        input_ids=batch['input_ids'].to('cuda'),
        max_length=max_target_length,
        attention_mask=batch['attention_mask'].to('cuda'),
        return_dict_in_generate=True, output_attentions=True)
    with tokenizer.as_target_tokenizer():
        outputs = [tokenizer.decode(out, clean_up_tokenization_spaces=False, skip_special_tokens=True) for out in outputs.sequences]
        labels = np.where(batch['labels'] != -100,  batch['labels'], tokenizer.pad_token_id)
        actuals = [tokenizer.decode(out, clean_up_tokenization_spaces=False, skip_special_tokens=True) for out in labels]
    
    
    #post-processing
    
    #outputs = [postprocessing(t) for t in outputs]
    
    predictions.extend(outputs)
    references.extend(actuals)
    metrics.add_batch(predictions=outputs, references=actuals)

metrics.compute()


In [ ]:
[{k: v.mid.fmeasure} for k,v in metrics.compute(predictions=predictions, references=references).items()]

In [ ]:
from evaluation.bleu import Bleu
from evaluation.cider import Cider

import numpy as np
import json

def compute_scores(gts, gens):
    bleu = Bleu()
    cider = Cider()
    bleu_scores, _ = bleu.compute_score(gts, gens)
    avg_bleu_score = np.array(bleu_scores).mean()
    cider_score, _ = cider.compute_score(gts, gens)

    return {
        "BLEU-1": bleu_scores[0],
        "BLEU-2": bleu_scores[1],
        "BLEU-3": bleu_scores[2],
        "BLEU-4": bleu_scores[3], 
        "BLEU": avg_bleu_score, 
        "CIDEr": cider_score
    }

def evaluate(truth_file: str, submission_file, code_phase) -> dict:
    gts = json.load(open(truth_file))
    gts = {id: [answer] for id, answer in gts.items()}

    gens = json.load(open(submission_file))
    gens = {id: [answer] for id, answer in gens.items()}

    if code_phase == "public_test":
        return compute_scores(gts, gens)
    else:
        gens = {id: answer for id, answer in gens.items() if id in gts}
        return compute_scores(gts, gens)

In [ ]:
dev_groundtruth = {k:i['labels'] for k,i in zip(dev_json['annotations'].keys(),dev_dataset)}
dev_predicted = {k:i for k,i in zip(dev_json['annotations'].keys(),predictions)}

In [ ]:
import json

with open('submission.json', 'w') as fp:
    json.dump(dev_predicted, fp,ensure_ascii=True,indent=True)

with open('vls2023_openvivqa_dev_answers.json', 'w') as fp:
    json.dump(dev_groundtruth , fp,ensure_ascii=True, indent=True)

In [ ]:
import pandas as pd
r = evaluate('/kaggle/working/vls2023_openvivqa_dev_answers.json',
         '/kaggle/working/submission.json',
         'public_test')
r = {k:[v] for k,v in r.items()}
df_result = pd.DataFrame(data=r)
html = df_result.style.set_table_styles([{'selector': 'th', 'props': [('font-size', '13pt')]}]).set_properties(**{'font-size': '13pt'}).format('{:.4f}')
html

In [ ]:
!zip -r checkpoint_5300.zip /kaggle/working/checkpoint/checkpoint-5300

In [ ]:
from IPython.display import FileLink
FileLink(r'checkpoint_5300.zip')

In [ ]:
from PIL import Image
from IPython.display import Image, display
import random

dev_set_pred = dev_set.copy().reset_index().drop('index',axis=1)
dev_set_pred['preds'] = dev_predicted.values()

def show_example():
    a_sample = dev_set_pred.sample(1)
    
    q = a_sample['question'].values[0]
    gt = a_sample['answer'].values[0]
    img = a_sample['image_id'].values[0]
    p = a_sample['preds'].values[0]
    
    img_p = '/kaggle/working/images/' + img_dict[str(img)]
    display(Image(img_p, width=300))
    print(a_sample.index[0])
    
    print('Image id: ',img)
    print('Question:',q)
    print('Groundtruth:',gt)
    print("Predicted:",p)
    #print('Metrics: ',result[a_sample.index[0]])

show_example()

In [ ]:
def predict(img,q):
    
    q1 = tokenizer.encode(q,return_tensors='pt')
    attention_mask = torch.ones_like(q1)
    
    outputs = model.generate(image_id = img,
        input_ids=q1.to('cuda'),
        max_length=50,
        attention_mask=attention_mask.to('cuda'),
        return_dict_in_generate=True, output_attentions=True)
    with tokenizer.as_target_tokenizer():
        outputs1 = [tokenizer.decode(out, clean_up_tokenization_spaces=False, skip_special_tokens=True) for out in outputs.sequences]
    
    out = model(input_ids=q1.cuda(),image_id = img,
                attention_mask=attention_mask.to('cuda'),
                decoder_input_ids=outputs.sequences, output_attentions=True, return_dict=True)

    encoder_attentions = out.encoder_attentions
    cross_attentions = out.cross_attentions
    decoder_attentions = out.decoder_attentions
    return q1,outputs1, cross_attentions

In [ ]:
from PIL import Image
from IPython.display import display

q = 'đây là đâu'
idx = random.choice(list(img_dict.keys()))
#idx ='8336'
#idx = '4179'
#idx = '111'
#idx = '8701'
#idx = '5775'
#idx = '3717'
print(idx)
img = [idx]

display(Image.open('/kaggle/working/images/'+img_dict[img[0]]))
q,a,attn = predict(img,q)
q = [tokenizer.convert_ids_to_tokens(i) for i in q[0].tolist()]
print(a)

import cv2
import matplotlib.pyplot as plt

for i in range(100):
    plt.figure()
    try:
        attn_img = attn[-1][0][-1,i,:196].detach().cpu()
    except:
        break
    attn_img = torch.reshape(attn_img , (-1,14)).cpu()


    tmp = cv2.imread('/kaggle/working/images/'+img_dict[img[0]])
    tmp = cv2.resize(np.array(tmp), (224,224))
    heatmap = cv2.resize(np.array(attn_img) , (224,224))
    heatmapshow = None
    heatmapshow = cv2.normalize(heatmap, heatmapshow, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    heatmapshow = cv2.applyColorMap(heatmapshow, cv2.COLORMAP_JET)

    final = cv2.addWeighted(src1=tmp,alpha=0.5,src2=heatmapshow,beta=0.5,gamma=0)

    plt.imshow(final)

In [ ]:
dev_predicted

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
attn[-1][-1][-1,:-1,:196].size()#.size()
for i in range(12):
    sns.heatmap(attn[-1][0][i,:,196:].detach().cpu(),cmap='BuPu')
    plt.show()

In [ ]:
while True:
    print(60)
    time.sleep(60)